In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()
LetsPlot.set_theme(theme_gray())

In [3]:
df = pd.read_csv("data/usgs_earthquakes_2025_m45.csv", parse_dates=["date"])
print(df.shape)
df.head(2)

(8252, 22)


,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,date
0,-3.8682,151.6536,10.0,4.7,mb,39.0,99.0,0.557,0.86,us,...,"67 km WNW of Rabaul, Papua New Guinea",earthquake,6.86,1.901,0.108,26.0,reviewed,us,us,2025-01-01
1,-3.8521,151.6207,10.0,4.5,mb,36.0,98.0,0.591,0.98,us,...,"71 km WNW of Rabaul, Papua New Guinea",earthquake,7.80,1.901,0.116,22.0,reviewed,us,us,2025-01-01


In [4]:
# histogram custom breaks

magnitude_histogram_plot = ggplot(df, aes(x="mag")) + \
    geom_histogram(breaks=[4.5, 4.6, 4.8, 5, 5.5, 9]) + \
    ggtitle("Magnitude distribution")

depth_histogram_plot = ggplot(df, aes(x="depth")) + \
    geom_histogram(breaks=[0, 10, 40, 100, 400, 700]) + \
    ggtitle("Depth distribution")

gggrid([magnitude_histogram_plot, depth_histogram_plot])

In [5]:
# pointdensity

p = ggplot(df.sort_values(by="mag"), aes("longitude", "latitude")) + \
    geom_livemap() + \
    theme(legend_position='bottom')
    
gggrid([
    p + \
        geom_point(aes(color="mag")) + \
        scale_color_viridis(option='magma') + \
        ggtitle("Epicenters: magnitude"),
    p + \
        geom_pointdensity() + \
        scale_color_viridis() + \
        ggtitle("Epicenters: density")
], ncol=1) + ggsize(800, 1000)

In [6]:
# legend wraps automatically
# ggtb(size_zoomin=...)

depth_vs_magnitude_plot = ggplot(df, aes("mag", "depth", color="magType")) + \
    geom_point(alpha=.5) + \
    theme(legend_position='bottom') + \
    ggtitle("Depth vs. Magnitude") + \
    ggtb(size_zoomin=2)
depth_vs_magnitude_plot

In [7]:
# facet_wrap(drop=...)
# theme(panel_spacing=..., strip_spacing=...)

ggplot(df.assign(depth_big=(df["depth"] >= 350).map({True: "depth >= 350", False: "depth < 350"}))) + \
    geom_point(aes("mag", "depth", fill="magType"), shape=21, alpha=.5, tooltips='none') + \
    facet_wrap(["magType", "depth_big"], drop=False, ncol=2) + \
    theme(panel_spacing=4, strip_spacing=2)

In [8]:
# geom_qq() preserve the mapping to original data after statistical transformation

magnitude_qq_plot = ggplot(df, aes(sample="mag")) + \
    geom_qq(distribution='exp', alpha=.5,
            tooltips=layer_tooltips().line("@|@magType").line("@|@mag").line("@|@depth")) + \
    ggtitle("Q-Q plot of magnitudes", "distribution='exp'")
magnitude_qq_plot

In [9]:
# shared legends

def get_date_month_mean_df(df, col):
    return df.groupby([df["date"], df["date"].dt.month.rename("month")])[col].mean().to_frame().reset_index()

month_colors = [                      "#CFE9FF",
                "#9BE06A", "#64D48C", "#38C172",
                "#14B85A", "#00A86B", "#00A0A8",
                "#FFD400", "#FF7A1A", "#FF3B30",
                "#B9C0C8", "#A7D8FF"]

gggrid([
    ggplot(get_date_month_mean_df(df, "mag")) + \
        geom_line(aes("date", "mag", color="month")) + \
        scale_color_gradientn(colors=month_colors),
    ggplot(get_date_month_mean_df(df, "depth")) + \
        geom_line(aes("date", "depth", color="month")) + \
        scale_color_gradientn(colors=month_colors),
], guides='collect') + \
    theme(legend_position='bottom') + \
    ggtb()

In [10]:
# sina & group=[]